<a href="https://colab.research.google.com/github/laribar/CONTADORLOGISTICO/blob/main/Treino_de_modeloS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.8/89.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 135.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
import av
import cv2
import numpy as np
import streamlit as st
import os
from collections import Counter
from ultralytics import YOLO
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase, WebRtcMode, RTCConfiguration
import tempfile

# Configuração do Streamlit
st.set_page_config(page_title="Contador de Objetos", layout="wide")
st.title("📷 Contador de Objetos — Streamlit com YOLOv8")

# --- Carregar o SEU MODELO TREINADO ---
MODEL_PATH = "best_pecas_pequenas_otimizado.pt"  # Seu modelo excelente!

# Verificar se o modelo existe
if not os.path.exists(MODEL_PATH):
    st.error(f"❌ Modelo não encontrado: {MODEL_PATH}")
    st.stop()

# Carregar modelo
@st.cache_resource
def load_model():
    return YOLO(MODEL_PATH)

yolo_model = load_model()
st.sidebar.success(f"✅ Modelo carregado: mAP50 98.9%")

# --- CONFIGURAÇÕES OTIMIZADAS para seu modelo ---
st.sidebar.header("Configurações YOLO")

# ⚠️ CRÍTICO: Usar os MESMOS parâmetros da validação
conf_thres = st.sidebar.slider("Confiança mínima", 0.1, 0.9, 0.25, 0.05)  # 0.25 como na validação!
iou_thres = st.sidebar.slider("IoU NMS", 0.1, 0.9, 0.50, 0.05)  # Ajustado
max_det = st.sidebar.slider("Máximo de detecções", 50, 1000, 300, 10)

draw_boxes = st.sidebar.checkbox("Desenhar anotações", True)
show_fps = st.sidebar.checkbox("Mostrar FPS", True)

# === Seleção da Fonte ===
source_option = st.sidebar.selectbox(
    "Escolher Fonte de Mídia",
    ["Webcam (Live Stream)", "Carregar Imagem da Galeria", "Carregar Vídeo Local"]
)

uploaded_file = None
if source_option == "Carregar Imagem da Galeria":
    uploaded_file = st.sidebar.file_uploader("Carregue uma imagem", type=["png", "jpg", "jpeg"])
elif source_option == "Carregar Vídeo Local":
    uploaded_file = st.sidebar.file_uploader("Carregue um vídeo", type=["mp4", "avi", "mov"])

# --- Configuração WebRTC SIMPLIFICADA ---
RTC_CONFIG = RTCConfiguration({"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]})

# =========================================================================
# === TRANSFORMER OTIMIZADO para seu modelo ===
# =========================================================================
class ObjectDetector(VideoTransformerBase):
    def __init__(self, model):
        self.model = model
        self.counter = Counter()
        self.frame_count = 0

    def transform(self, frame: av.VideoFrame) -> np.ndarray:
        try:
            # Converter frame para numpy
            image = frame.to_ndarray(format="bgr24")

            # Fazer predição
            results = self.model.predict(
                source=image,
                conf=conf_thres,  # ⚠️ USA A CONFIGURAÇÃO DO SLIDER!
                iou=iou_thres,
                verbose=False,
                max_det=max_det
            )

            # Processar resultados
            detections = results[0]
            self.counter = Counter()

            if detections.boxes is not None and len(detections.boxes) > 0:
                boxes = detections.boxes.xyxy.cpu().numpy().astype(int)
                classes = detections.boxes.cls.cpu().numpy().astype(int)
                confidences = detections.boxes.conf.cpu().numpy()
                class_names = detections.names

                for (x1, y1, x2, y2), class_id, conf in zip(boxes, classes, confidences):
                    class_name = class_names.get(class_id, str(class_id))
                    self.counter[class_name] += 1

                    if draw_boxes:
                        # Desenhar caixa
                        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                        # Texto com classe e confiança
                        label = f"{class_name} {conf:.2f}"
                        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                        cv2.rectangle(image, (x1, y1 - label_size[1] - 10),
                                    (x1 + label_size[0], y1), (0, 255, 0), -1)
                        cv2.putText(image, label, (x1, y1 - 5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Adicionar contadores na imagem
            y_offset = 35
            for obj_type, count in sorted(self.counter.items()):
                text = f"{obj_type}: {count}"
                cv2.putText(image, text, (10, y_offset),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                y_offset += 30

            # Adicionar FPS
            if show_fps:
                cv2.putText(image, "FPS: Live", (image.shape[1] - 120, 30),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

            self.frame_count += 1
            return image

        except Exception as e:
            # Em caso de erro, retorna imagem com mensagem
            error_img = np.zeros((480, 640, 3), dtype=np.uint8)
            cv2.putText(error_img, f"Erro: {str(e)}", (10, 30),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            return error_img

# =========================================================================
# === LÓGICA PRINCIPAL ===
# =========================================================================

if source_option == "Webcam (Live Stream)":
    st.info("🎥 **Webcam ao Vivo** - Clique em 'START' para iniciar")

    webrtc_ctx = webrtc_streamer(
        key="object-counter",
        mode=WebRtcMode.SENDRECV,
        rtc_configuration=RTC_CONFIG,
        media_stream_constraints={
            "video": {
                "width": {"ideal": 640},
                "height": {"ideal": 480},
                "frameRate": {"ideal": 20}
            },
            "audio": False
        },
        video_processor_factory=lambda: ObjectDetector(yolo_model),
        async_processing=True,
    )

    if webrtc_ctx.state.playing:
        st.success("✅ Câmera ativa - Detecção funcionando!")
    else:
        st.warning("⏸️ Clique em START para iniciar a câmera")

elif uploaded_file is not None:
    if source_option == "Carregar Imagem da Galeria":
        try:
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)

            if img is not None:
                # Processar imagem
                results = yolo_model.predict(
                    source=img,
                    conf=conf_thres,
                    iou=iou_thres,
                    verbose=False
                )

                # Plotar resultados
                plotted_img = results[0].plot()
                st.image(plotted_img, channels="BGR",
                        caption="🔍 Detecção na Imagem",
                        use_column_width=True)

                # Mostrar contagem
                counter = Counter()
                if results[0].boxes is not None:
                    for class_id in results[0].boxes.cls.cpu().numpy().astype(int):
                        class_name = results[0].names[class_id]
                        counter[class_name] += 1

                if counter:
                    count_text = " | ".join([f"**{k}**: {v}" for k, v in counter.items()])
                    st.success(f"📊 **Contagem:** {count_text}")
                else:
                    st.warning("⚠️ Nenhum objeto detectado")
            else:
                st.error("❌ Erro ao carregar imagem")

        except Exception as e:
            st.error(f"❌ Erro: {str(e)}")

# =========================================================================
# === STATUS DO MODELO ===
# =========================================================================

st.sidebar.markdown("---")
st.sidebar.markdown("### 📊 Status do Modelo")
st.sidebar.success(f"**mAP50:** 98.9%")
st.sidebar.info(f"**Melhor época:** 36")
st.sidebar.info(f"**Classes:** Bolt, Nut, Washer")

st.markdown("---")
st.markdown("""
### 🎯 **Instruções para Webcam:**

1. **Clique em START** na janela da webcam
2. **Ajuste a confiança para 0.25** (igual ao treino)
3. **Mostre peças próximas** à câmera (parafusos, porcas, arruelas)
4. **Boa iluminação** é essencial

### ⚠️ **Se ainda não detectar:**
- Verifique **permissões da câmera** no navegador
- Tente modo **incógnito**
- Teste com **imagem estática** primeiro
""")

loading Roboflow workspace...
loading Roboflow project...
=== ANÁLISE DO DATASET ===
Classes: ['Bolt', 'Nut', 'washer']
Número de classes: 3
Imagens de treino: 0
Imagens de validação: 0
Usando modelo: yolov8n.pt
Iniciando treinamento otimizado...
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Nuts-and-Bolts-2/data.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.01, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mas

FileNotFoundError: ../test_image.jpg does not exist

In [5]:
# ==========================================================
# PASSO 1: DOWNLOAD E ANÁLISE DO DATASET
# ==========================================================
from roboflow import Roboflow
from ultralytics import YOLO
import shutil, os
import yaml
from collections import Counter

# Credenciais e Projeto
rf = Roboflow(api_key="wvg2BzupQ0kIkj0IFSt9")
project = rf.workspace("kiproject-8z6ry").project("screw-nail-dowel-project")
version = project.version(3)

# Baixa o dataset
dataset = version.download("yolov8")
data_yaml_path = os.path.join(dataset.location, 'data.yaml')

# ==========================================================
# NOVO: ANÁLISE DETALHADA DO DATASET
# ==========================================================
def analyze_dataset_comprehensive(data_yaml_path):
    """Análise completa do dataset para identificar problemas"""
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    print("🔍 === ANÁLISE COMPLETA DO DATASET ===")
    print(f"Classes: {data['names']}")
    print(f"Número de classes: {data['nc']}")

    # Contar imagens e anotações
    total_images = 0
    total_annotations = 0
    class_distribution = Counter()

    for split in ['train', 'val']:
        if split in data and os.path.exists(data[split]):
            image_files = []
            with open(data[split], 'r') as f:
                image_files = [line.strip() for line in f if line.strip()]

            print(f"\n📁 {split.upper()}:")
            print(f"  Imagens: {len(image_files)}")

            split_annotations = 0
            for img_path in image_files[:10]:  # Amostra das primeiras 10
                label_path = img_path.replace('images', 'labels').replace('.jpg', '.txt')
                if os.path.exists(label_path):
                    with open(label_path, 'r') as lbl:
                        annotations = [line.strip() for line in lbl if line.strip()]
                        split_annotations += len(annotations)
                        for ann in annotations:
                            class_id = int(ann.split()[0])
                            class_distribution[class_id] += 1

            total_images += len(image_files)
            total_annotations += split_annotations
            print(f"  Anotações (amostra): {split_annotations}")

    print(f"\n📊 DISTRIBUIÇÃO DE CLASSES:")
    for class_id, count in class_distribution.most_common():
        class_name = data['names'].get(class_id, f"Classe_{class_id}")
        print(f"  {class_name}: {count} anotações")

    return data, total_images, total_annotations

# Executar análise
dataset_info, total_imgs, total_anns = analyze_dataset_comprehensive(data_yaml_path)

# ==========================================================
# PASSO 2: TREINAMENTO OTIMIZADO
# ==========================================================

# 1. Escolha inteligente do modelo base baseado no dataset
if total_imgs < 500:  # Dataset pequeno
    model_choice = 'yolov8n.pt'  # Nano - menos overfitting
    print("📦 Dataset pequeno detectado. Usando YOLOv8n...")
else:
    model_choice = 'yolov8s.pt'  # Small - balanceado
    print("📦 Dataset médio/grande. Usando YOLOv8s...")

model = YOLO(model_choice)

# 2. Parâmetros OTIMIZADOS para objetos pequenos
print("🚀 Iniciando treinamento otimizado...")
results = model.train(
    data=data_yaml_path,
    epochs=100,              # Aumentado para melhor convergência
    imgsz=416,               # ⚠️ CRÍTICO: Reduzido para objetos pequenos!
    batch=16,                # Balanceado para VRAM
    lr0=0.01,                # Taxa de aprendizado
    patience=15,             # Early stopping
    augment=True,            # Aumentação básica
    # 🔥 AUMENTAÇÃO AVANÇADA para objetos pequenos:
    hsv_h=0.015,            # Variação de matiz
    hsv_s=0.7,              # Variação de saturação
    hsv_v=0.4,              # Variação de brilho
    degrees=10.0,           # Rotação aumentada
    translate=0.2,          # Translação
    scale=0.5,              # Escala
    shear=5.0,              # Cisalhamento
    perspective=0.0005,     # Perspectiva
    flipud=0.1,             # Flip vertical
    fliplr=0.5,             # Flip horizontal
    mosaic=0.8,             # Mosaic augmentation
    mixup=0.1,              # Mixup augmentation
    copy_paste=0.1,         # Copy-paste
    erasing=0.3,            # Random erasing
    # ⚙️ OTIMIZAÇÕES ESPECÍFICAS:
    close_mosaic=10,        # Desativa mosaic nas últimas épocas
    overlap_mask=True,      # Melhor para objetos próximos
    mask_ratio=4,           # Resolução de máscara
    dropout=0.1,            # Regularização
    name='parafuso_prego_treino_otimizado_v2'
)

print("✅ Treinamento concluído!")

# ==========================================================
# PASSO 3: VALIDAÇÃO DETALHADA
# ==========================================================

best_pt_path = 'runs/detect/parafuso_prego_treino_otimizado_v2/weights/best.pt'

if os.path.exists(best_pt_path):
    print("\n📊 === VALIDAÇÃO DO MODELO ===")

    # Carregar modelo treinado
    model_trained = YOLO(best_pt_path)

    # Validação com métricas detalhadas
    metrics = model_trained.val(
        split='val',
        conf=0.25,      # Confiança para validação
        iou=0.50,       # IoU para objetos pequenos
        save_json=True,  # Salva métricas em JSON
        plots=True       # Gera gráficos
    )

    # 📈 RELATÓRIO DE MÉTRICAS
    print(f"\n🎯 MÉTRICAS DE DESEMPENHO:")
    print(f"   mAP@50: {metrics.box.map50:.3f}")
    print(f"   mAP@50-95: {metrics.box.map:.3f}")
    print(f"   Precisão: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")

    # 📝 CLASSES INDIVIDUAIS
    if hasattr(metrics.box, 'maps'):
        print(f"\n📋 MÉTRICAS POR CLASSE:")
        for i, class_name in enumerate(dataset_info['names'].values()):
            if i < len(metrics.box.maps):
                print(f"   {class_name}: {metrics.box.maps[i]:.3f}")

    # 🧪 TESTE PRÁTICO
    print(f"\n🔍 TESTE PRÁTICO:")
    test_results = model_trained.predict(
        source=os.path.join(dataset.location, 'valid/images'),  # Usa validação
        conf=0.3,
        save=True,
        save_txt=True
    )

    # 💾 SALVAR MODELO FINAL
    destino_final = 'best_parafuso_prego_otimizado.pt'
    shutil.copy(best_pt_path, destino_final)

    print(f"\n🎉 MODELO OTIMIZADO PRONTO!")
    print(f"   📁 Salvo em: {destino_final}")
    print(f"   📈 mAP@50: {metrics.box.map50:.3f}")

    # ✅ AVALIAÇÃO DE QUALIDADE
    if metrics.box.map50 >= 0.75:
        print("   🏆 EXCELENTE: Modelo de alta qualidade!")
    elif metrics.box.map50 >= 0.50:
        print("   👍 BOM: Modelo utilizável")
    else:
        print("   ⚠️ MODERADO: Pode precisar de ajustes")

else:
    print("❌ ERRO: Modelo treinado não encontrado!")

# ==========================================================
# PASSO 4: RECOMENDAÇÕES PARA STREAMLIT
# ==========================================================

print(f"\n🔧 CONFIGURAÇÃO PARA STREAMLIT:")
print(f"   Modelo: best_parafuso_prego_otimizado.pt")
print(f"   conf_thres: 0.25 - 0.35")
print(f"   iou_thres: 0.45")
print(f"   ⚠️ Lembre-se: Objetos pequenos precisam de confiança mais baixa!")

loading Roboflow workspace...
loading Roboflow project...
🔍 === ANÁLISE COMPLETA DO DATASET ===
Classes: ['Dowel', 'Nail', 'Screw']
Número de classes: 3

📊 DISTRIBUIÇÃO DE CLASSES:
📦 Dataset pequeno detectado. Usando YOLOv8n...
🚀 Iniciando treinamento otimizado...
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Screw,-Nail-&-Dowel-Project-3/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.3, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_widt

AttributeError: 'list' object has no attribute 'values'